In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

# 데이터 로드
data = pd.read_csv("../../../../monunmon.csv")  # CSV 파일 경로

# 레이블 이진 분류를 위해 변환 (0~94 -> 1, -1 -> 0)
data['Label'] = data['Label'].apply(lambda x: 1 if x >= 0 else 0)

# 변환 결과 확인
print("Label Distribution:")
print(data['Label'].value_counts())

# 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

# Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# 데이터 정규화 (Adaboost는 정규화 없이도 잘 작동하지만, 정규화를 적용해도 괜찮음)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Adaboost 모델 정의 및 학습
model = AdaBoostClassifier(n_estimators=100, random_state=42)  # n_estimators는 부스팅 단계 수
model.fit(X_train, y_train)

# 검증 데이터 평가
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)

print(f"Validation Accuracy: {val_accuracy:.2f}")
print(f"Validation F1 Score: {val_f1:.2f}")

# 테스트 데이터 평가
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test F1 Score: {test_f1:.2f}")

# 분류 보고서 출력
print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_test_pred))


Label Distribution:
Label
1    19000
0     3000
Name: count, dtype: int64
Validation Accuracy: 0.89
Validation F1 Score: 0.94
Test Accuracy: 0.89
Test F1 Score: 0.94

Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.75      0.26      0.38       450
           1       0.89      0.99      0.94      2850

    accuracy                           0.89      3300
   macro avg       0.82      0.62      0.66      3300
weighted avg       0.87      0.89      0.86      3300



In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.tree import DecisionTreeClassifier

# 데이터 로드
data = pd.read_csv("../../../../monunmon.csv")  # 데이터셋 경로 입력
data['Label'] = data['Label'].apply(lambda x: 1 if x >= 0 else 0)  # 이진 레이블 변환

# 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

# Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

### 방법 1: 클래스 가중치 적용 ###
print("=== AdaBoost with Class Weights ===")
base_estimator_weighted = DecisionTreeClassifier(max_depth=1, class_weight='balanced')  # 기본 학습기에 가중치 적용
model_weighted = AdaBoostClassifier(base_estimator=base_estimator_weighted, n_estimators=100, random_state=42)
model_weighted.fit(X_train, y_train)

# 테스트 데이터 평가
y_test_pred_weighted = model_weighted.predict(X_test)
print("\nClassification Report on Test Data (Class Weights):")
print(classification_report(y_test, y_test_pred_weighted))

### 방법 2: SMOTE를 이용한 오버샘플링 ###
print("\n=== AdaBoost with SMOTE ===")
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("Resampled Training Set Size (SMOTE):", X_train_smote.shape)

base_estimator_smote = DecisionTreeClassifier(max_depth=1)  # 기본 학습기 설정
model_smote = AdaBoostClassifier(base_estimator=base_estimator_smote, n_estimators=100, random_state=42)
model_smote.fit(X_train_smote, y_train_smote)

# 테스트 데이터 평가
y_test_pred_smote = model_smote.predict(X_test)
print("\nClassification Report on Test Data (SMOTE):")
print(classification_report(y_test, y_test_pred_smote))




=== AdaBoost with Class Weights ===


c:\Users\yzooz\pythonenv\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(



Classification Report on Test Data (Class Weights):
              precision    recall  f1-score   support

           0       0.32      0.69      0.44       450
           1       0.94      0.77      0.85      2850

    accuracy                           0.76      3300
   macro avg       0.63      0.73      0.64      3300
weighted avg       0.86      0.76      0.79      3300


=== AdaBoost with SMOTE ===
Resampled Training Set Size (SMOTE): (26600, 15)


c:\Users\yzooz\pythonenv\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(



Classification Report on Test Data (SMOTE):
              precision    recall  f1-score   support

           0       0.33      0.71      0.45       450
           1       0.94      0.77      0.85      2850

    accuracy                           0.76      3300
   macro avg       0.64      0.74      0.65      3300
weighted avg       0.86      0.76      0.79      3300

